In [1]:
#!pip install python-dotenv
#!pip install openai

In [2]:
import os
import openai

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


In [24]:
def get_completion(prompt, model=llm_model):
    messsages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messsages,
        temperature=0,
    )
    return response.choices[0].message["content"]


In [25]:
get_completion("Tell me a joke about AI", model=llm_model)

'Why did the AI break up with its robot girlfriend? Because she had too many bugs in her system!'

In [12]:
customer_email = """Can you please check whether my gift card is still valid? \
    Thank you so much! I really appreciate your support! Do not worry I can wait for a few days. I know you are busy!"""

In [13]:
style = """British English in a professional tone"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple dollar signs into a style that is {style}.
text: $$${customer_email}$$$
"""
print(prompt)

Translate the text that is delimited by triple dollar signs into a style that is British English in a professional tone.
text: $$$Can you please check whether my gift card is still valid?     Thank you so much! I really appreciate your support! Do not worry I can wait for a few days. I know you are busy!$$$



In [18]:
response = get_completion(prompt, model=llm_model)
response

'Could you please verify if my gift card is still valid? Thank you very much! I truly appreciate your assistance! Please do not be concerned, I am able to wait a few days. I understand that you have a busy schedule.'

In [19]:
#pip install --upgrade langchain

In [ ]:
from langchain.chat_models import ChatOpenAI

In [31]:
chat = ChatOpenAI(
    model_name=llm_model,
    temperature=0,
)


In [33]:
template_string = """Translate the text \
    that is delimited by triple dollar signs into a style that is {style}.
    text: $$${text}$$$"""


In [41]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [42]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text     that is delimited by triple dollar signs into a style that is {style}.\n    text: $$${text}$$$')

In [46]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [49]:
style = "British English in a professional tone"
customer_email = """Can you please check whether my gift card is still valid? I dont know if it is still valid because I have not used it for a while. \
    Thank you so much! I really appreciate your support! Do not worry I can wait for a few days. I know you are busy!"""
 

In [50]:
customer_message = prompt_template.format_messages(
    style=style,
    text=customer_email,
)

In [52]:
customer_message

[HumanMessage(content='Translate the text     that is delimited by triple dollar signs into a style that is British English in a professional tone.\n    text: $$$Can you please check whether my gift card is still valid? I dont know if it is still valid because I have not used it for a while.     Thank you so much! I really appreciate your support! Do not worry I can wait for a few days. I know you are busy!$$$')]

In [58]:
customer_message[0].content

'Translate the text     that is delimited by triple dollar signs into a style that is British English in a professional tone.\n    text: $$$Can you please check whether my gift card is still valid? I dont know if it is still valid because I have not used it for a while.     Thank you so much! I really appreciate your support! Do not worry I can wait for a few days. I know you are busy!$$$'

In [59]:
service_reply = """Dear Customer, Thank you for reaching out to us. We appreciate your patience and understanding. I will check the status of your gift card and get back to you as soon as possible. Thank you for your support! Best regards, Customer Service Team"""
service_style_pirate = "A polite tone that speaks in English Pirate"

In [ ]:
service_message = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply,
)
service_message[0].content

'Translate the text     that is delimited by triple dollar signs into a style that is A polite tone that speaks in English Pirate.\n    text: $$$Dear Customer, Thank you for reaching out to us. We appreciate your patience and understanding. I will check the status of your gift card and get back to you as soon as possible. Thank you for your support! Best regards, Customer Service Team$$$'

In [63]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [84]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [91]:
prompt_template = ChatPromptTemplate.from_template(review_template)
# print(prompt_template.messages[0].prompt)

messages = prompt_template.format_messages(
    text=customer_review,
)
# print(messages[0].content)

# tmp_model = "gpt-4o"
chat = ChatOpenAI(
    model_name=llm_model,
    # model_name=tmp_model,
    temperature=0,
)

response = chat(messages)
response.content

'{\n    "gift": true,\n    "delivery_days": 2,\n    "price_value": "It\'s slightly more expensive than the other leaf blowers out there"\n}'

In [94]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [127]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1.",
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list.",
)
response_schemas = [
    gift_schema,
    delivery_days_schema,
    price_value_schema,
]

In [121]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [132]:
print(type(output_parser))
output_parser


<class 'langchain.output_parsers.structured.StructuredOutputParser'>


StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.', type='string'), ResponseSchema(name='price_value', description='Extract any sentences about the value or price, and output them as a comma separated Python list.', type='string'), ResponseSchema(name='delivery_days', description='How many days did it take for the product to arrive? If this information is not found, output -1.', type='string')])

In [115]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\t"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.\n\t"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.\n}\n```'

In [123]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [124]:
messages[0].content

'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "

In [125]:
response = chat(messages)
response.content

'```json\n{\n\t"gift": false,\n\t"delivery_days": 2,\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [131]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': False,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [134]:
output_dict.get("delivery_days")    

2